<a href="https://colab.research.google.com/github/AdarshSingh1311/BACKTEST-1/blob/main/Google%20colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta
!pip install pandas_ta
!pip install --upgrade breeze-connect==1.0.40

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime , timedelta
from dateutil.relativedelta import relativedelta,TH , TU
import warnings
warnings.filterwarnings('ignore')
import ta
import pandas_ta as ta
import matplotlib.pyplot as plt
from breeze_connect import BreezeConnect

# Initialize SDK
breeze = BreezeConnect(api_key="f65&396497Vq0161W2d18ngV35%5755@")

# Obtain your session key from https://api.icicidirect.com/apiuser/login?api_key=YOUR_API_KEY
# Incase your api-key has special characters(like +,=,!) then encode the api key before using in the url as shown below.
import urllib
print("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus("your_api_key"))

# Generate Session
breeze.generate_session(api_secret="5l88!0708cJ04~74!X1V71a90j4~392+",
                        session_token="22545800")


In [ ]:
stock = 'TCS'
emas = 5
emal = 13
rsi_p = 14
rsi_u = 70
rsi_l = 30

trade = 0

trade_log = []
trade_detail = []
trade_date = []
trade_c = []


In [ ]:
breeze.ws_connect()

# Callback to receive ticks.
def on_ticks(ticks):
    #print("Ticks: {}".format(ticks))
    current_price = ticks['last']
    open_price = ticks['open']
    high_price = ticks['high']
    low_price = ticks['low']
    print(current_price , open_price , high_price , low_price)
    #print(trade)

    if datetime.now() >= MARKET_TRIG:
        print('pop')
        if  trade  == 0 :
            c_ema1 , c_ema2 , p_ema1 , p_ema2 , c_rsi , p_rsi ,m = get_ema_rsi( current_price , high_price , low_price , open_price )
            #print(c_ema1)
            if signal(c_ema1 , c_ema2 , p_ema1 , p_ema2 , c_rsi , p_rsi , trade , m) == 1 :
                #place_buy_order()
                print('place_buy_order')
        #print('lll')
        if  trade == 1 :
            if signal() == -1 :
                #buy_square_off ()
                print('buy_square_off')
                #place_sell_order()
                print('place_sell_order')

        if  trade == 0 :
            if signal(c_ema1 , c_ema2 , p_ema1 , p_ema2 , c_rsi , p_rsi , trade , m) == -1 :
                #place_sell_order()
                print('place_sell_order')

        if  trade == -1 :
            if signal(c_ema1 , c_ema2 , p_ema1 , p_ema2 , c_rsi , p_rsi , trade , m) == 1 :
                #sell_square_off()
                print('sell_square_off')
                #place_buy_order()
                print('place_buy_order')

        if  trade == 1 :
            if signal(c_ema1 , c_ema2 , p_ema1 , p_ema2 , c_rsi , p_rsi , trade , m) == 0 :
                #place_sell_order()
                print('place_sell_order')

        if  trade == -1 :
            if signal(c_ema1 , c_ema2 , p_ema1 , p_ema2 , c_rsi , p_rsi , trade , m) == 0 :
                #place_buy_order()
                print('place_buy_order')

        #no trade
        if trade == 0 :
            if signal(c_ema1 , c_ema2 , p_ema1 , p_ema2 , c_rsi , p_rsi , trade , m) == 0 :
                print('NO TRADES')


breeze.on_ticks = on_ticks

In [ ]:
breeze.subscribe_feeds(exchange_code="NSE",

                       stock_code="TCS",
                       product_type="cash",
                       get_exchange_quotes=True,
                       get_market_depth=False)

In [ ]:
def get_ema_rsi( current_price , high_price , low_price , open_price ) :
    now = datetime.now()
    from_date = now - timedelta(days = 40)
    now = now.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    from_date = from_date.strftime('%Y-%m-%dT%H:%M:%S.000Z')

    try :
        n = breeze.get_historical_data_v2(interval="1day",
                            from_date=  from_date,
                            to_date=  now,
                            stock_code= stock,
                            exchange_code="NSE",
                            product_type="cash")

        if (n['Status'] == 200) :
            m =pd.DataFrame(n['Success'])
            m.index = m.datetime
            m.drop( columns = {'datetime' , 'exchange_code' , 'stock_code' , 'volume' }  , inplace = True)
            m.rename( {'datetime' : 'date'} , inplace = True)
            # ADD TODAY'S LAST TICK DATA TO DATAFRAME
            m.loc[datetime.now().strftime('%Y-%m-%d') + ' ' +  datetime.now().strftime('%H:%M:%S')]= [current_price , high_price , low_price , open_price ]
            #print(emas,emal,rsi_p)
            m['EMAS'] = ta.ema(m['close'], length=emas)
            m['EMAL'] = ta.ema(m['close'], length=emal)

            m['RSI'] = ta.rsi(m['close'] ,length = rsi_p)
            #print(m)
            c_ema1 =  m['EMAS'].iloc[-1]
            c_ema2  = m['EMAL'].iloc[-1]
            p_ema1 =  m['EMAS'].iloc[-2]
            p_ema2  = m['EMAL'].iloc[-2]

            c_rsi = m['RSI'].iloc[-1]
            p_rsi = m['RSI'].iloc[-2]

            return c_ema1 , c_ema2 , p_ema1 , p_ema2 , c_rsi , p_rsi , m
            #print(c_ema1 , c_ema2 , p_ema1 , p_ema_2 , c_rsi , p_rsi)

    except Exception as error :
        print('Failed Historical API Request' , error )
        return False


In [ ]:
def signal ( c_ema1 , c_ema2 , p_ema1 , p_ema2 , c_rsi , p_rsi , trade , m) :
    c_ema1 = c_ema1
    c_ema2 = c_ema2
    p_ema1 = p_ema1
    p_ema2 = p_ema2
    c_rsi = c_rsi
    p_rsi = p_rsi
    trade = trade
    m = m
    #c_ema1 , c_ema2 , p_ema1 , p_ema2 , c_rsi , p_rsi =  get_ema_rsi( current_price , high_price , low_price , open_price )
    #print(c_ema1,c_ema2,p_ema1 , p_ema2 , c_rsi , p_rsi)

    # long main entry exit
    #print('HELLO')
    #print(trade)
    if  (p_ema1 < p_ema2) & (c_ema1 > c_ema1) & ( trade == 0) :
        #print('ahead')
        trade = 1
        #trade_log.append( f' Long {stock}  at  {long_price} ' )
        trade_detail.append( f' Long Crossover')
        trade_date.append(m.index[-1])

    if (p_ema1 > p_ema2 ) & (c_ema1 < c_ema2) & ( trade == 1) :
        trade = -1
        #trade_log.append(f' Long Exit {stock}  at  {long_exit_price} ')
        #trade_log.append(f' Short {stock}  at  {short_price} ')
        trade_detail.append( f' Crossover reversal Exit')
        trade_detail.append( f' Short Crossover')
        trade_date.append(m.index[-1])
        trade_date.append(m.index[-1])

    # Short main entry exit
    if (p_ema1 > p_ema2) & (c_ema1 < c_ema2) & ( trade == 0) :
        trade = -1
        trade_c.append( [ -1 , t.index[i] ] )
        #trade_log.append(f' Short {stock}  at  {short_price} ')
        trade_detail.append( f' Short Crossover')
        trade_date.append(m.index[-1])

    if (p_ema1 < p_ema2) & (c_ema1 > c_ema2) & ( trade == -1) :
        trade = 1
        #trade_log.append(f' Short Exit {stock}  at  {short_exit_price} ')
        #trade_log.append(f' Long {stock}  at  {long_price} ')
        trade_detail.append( f' Crossover reversal Exit')
        trade_detail.append( f' Long Crossover  Exit')
        trade_date.append(m.index[-1])
        trade_date.append(m.index[-1])

    # Long trade re enrty exit
    if ( trade == 1) & (p_rsi > rsi_u ) & (c_rsi < rsi_u ) :
        trade = 0
        #trade_log.append(f' Long Exit {stock}  at  {long_exit_price} ')
        trade_detail.append( f' RSI BREACH EXIT')
        trade_date.append(m.index[-1])

    if (trade == 0)  & (p_rsi < rsi_u ) & (c_rsi > rsi_u ) :
        trade = 1
        #trade_log.append(f' Long {stock}  at  {long_price} ')
        trade_detail.append( f' RSI RETNRY LONG')
        trade_date.append(m.index[-1])

    # Short trade re enrty exit
    if  ( trade == -1) & (p_rsi < rsi_l ) & (c_rsi > rsi_l ) :
        trade = 0
        #trade_log.append(f' Short Exit {stock}  at  {short_exit_price} ')
        trade_detail.append( f' RSI BREACH EXIT')
        trade_date.append(m.index[-i])

    if (trade == 0)  & (p_rsi > rsi_l ) & (c_rsi < rsi_l ) :
        trade = -1
        #trade_log.append(f' Short {stock}  at  {short_price} ')
        trade_detail.append( f' RSI RETNRY SHORT')
        trade_date.append(m.index[-1])


    else :
        trade_c.append([ trade, m.index[-1] ] )
        #print('ok')

    #print('done')
    return trade


In [ ]:
MARKET_TRIG = datetime.now().replace(hour = 11 , minute = 57, second = 0 , microsecond = 0)
MARKET_CLOSE = datetime.now().replace(hour = 15 , minute = 25, second = 0 , microsecond = 0)

In [ ]:
def timer () :
    while(MARKET_CLOSE > datetime.now() > MARKET_TRIG  ) :
        print( ' TRADES GETTING EXECUTED')
        time.sleep(5000000000000)

    if  ( datetime.now() >   MARKET_CLOSE ) :

        breeze.unsubscribe_feeds(exchange_code="NSE",
                       stock_code="TCS",
                       product_type="cash",
                       get_exchange_quotes=True,
                       get_market_depth=False)

        api.ws_disconnect()


    while (  datetime.now()  <  MARKET_TRIG ) :
        print('NO EXECUTION OF TRADE')

In [ ]:
timer()

In [ ]:
def ttimer () :
  while True :
    pass

In [ ]:
ttimer()

In [ ]:
!pip install Connect
!pip install bidict
!pip install certifi
!pip install chardet
!pip install idna
!pip install requests
!pip install urllib3

In [ ]:
!pip install -U connect.py

In [ ]:
import configparser
import json
import os
import csv
from threading import Thread
#from MarketDataSocketClient import MDSocket_io


In [2]:
!pip install Connect  # Install the module

from Connect import XTSConnect

ModuleNotFoundError: ignored

In [ ]:
!pip install XTConnect


In [ ]:
from XTConnect.Connect import XTSConnect

In [1]:
pip install -r requirements.txt

In [11]:
#	[user]
#source=WEBAPI

#	[SSL]
disable_ssl=True

#	[root_url]
#root=https://developers.symphonyfintech.in

#accesspassword=2021HostLookUpAccess

#version=interactiveapi_1.0.1

#broadcastMode=Full

In [14]:
! pip install XTSConnect

ERROR: Could not find a version that satisfies the requirement XTSConnect (from versions: none)
ERROR: No matching distribution found for XTSConnect


In [12]:
"""API Credentials"""
API_KEY = "2d37c3e06ff898ba612277"
API_SECRET = "Wqqv072@I#"
XTS_API_BASE_URL = " http://ctrade.jainam.in:3000/"
source = "WEBAPI"

"""Make XTSConnect object by passing your interactive API appKey, secretKey and source"""
xt = XTSConnect(API_KEY, API_SECRET, source)


NameError: ignored

In [ ]:
breeze.ws_connect()

# Callback to receive ticks.
def on_ticks(ticks):
    print("Ticks: {}".format(ticks))

# Assign the callbacks.
breeze.on_ticks = on_ticks

In [ ]:
breeze.subscribe_feeds(exchange_code="NSE",
                       stock_code="TCS",
                       product_type="cash",
                       get_exchange_quotes=True,
                       get_market_depth=False)

In [ ]:
breeze.ws_disconnect()